In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import glob
import csv
import pandas as pd
import re

In [40]:
def measure_geo_distance(pt1_longitude,pt1_latitude,pt2_longtide,pt2_latitude):
    """
    This function compute the geo_distance between two points using Euclidean distance
    """
    distance = ((pt1_longitude-pt2_longtide)**2+(pt1_latitude-pt2_latitude)**2)**0.5
    return distance

In [26]:
# Read the Dataset
Data_Jan_Pickup=pd.read_csv('DC_Cab_Pickup/Jan_DC_Data.csv',sep = '\t')
#Data_Feb_Pickup=pd.read_csv('DC_Cab_Pickup/Feb_DC_Data.csv',sep = '\t')
#Data_Mar_Pickup=pd.read_csv('DC_Cab_Pickup/Mar_DC_Data.csv',sep = '\t')

# Read the Region Data Set
data_regions=pd.read_csv('DC_Cab_Pickup/CoordinateWrenches.csv')

In [87]:
data_pickup = Data_Jan_Pickup[:1000]

In [32]:
Data_Jan_Pickup[:1]

,Day_of_Pickupdatetime_Tr,Dropoff_Block_Latitude,Dropoff_Block_Longitude,Hour_of_Pickupdatetime_Tr,Month_of_Pickupdatetime_Tr,Pickup_Block_Latitude,Pickup_Block_Longitude,Count_of_Objectid
0,1,38.823601,-77.00889,0,January,38.823601,-77.00889,2


In [38]:
# Select the distance threshold - maximum distance of a trip
trip_distance_max = min(data_regions['Radius'])

In [45]:
data_regions

,Latitude,Longitude,Radius
0,38.903646,-77.052080,0.015497
1,38.897879,-77.026263,0.012657
2,38.905107,-77.036653,0.012657
3,38.893360,-77.008867,0.017973
4,38.921966,-77.047168,0.018967


In [105]:
def filter_by_origin(data_pickup,data_regions):
    """
    This function filter out the trips that are not originated from any of our selected region, and add a region tag
    data_pickup
    data_region
    """
    # Create a copy
    data_pickup_filtered = data_pickup.copy()

    # Initialize
    in_region_list = []
    in_which_region_list = []

    # Loop through each row
    for i in range(0, len(data_pickup)):
        pickup_latitude = data_pickup['Pickup_Block_Latitude'][i]
        pickup_longitude = data_pickup['Pickup_Block_Longitude'][i]
        in_region = 0
        in_which_region = -1
        for j in range(0,len(data_regions)):
            distance = measure_geo_distance(pickup_longitude,pickup_latitude,data_regions['Longitude'][j],data_regions['Latitude'][j])
            if distance <= data_regions['Radius'][j]:
                in_region = 1
                in_which_region = j
        in_region_list.append(in_region)
        in_which_region_list.append(in_which_region)
    data_pickup_filtered['Region'] = in_which_region_list
    data_pickup_filtered = data_pickup_filtered[data_pickup_filtered.Region != -1]
    return data_pickup_filtered

In [106]:
data_pickup_filtered = filter_by_origin(data_pickup,data_regions)

In [107]:
len(data_pickup_filtered)

892